# DECISION TREE IMPLEMENTATION

Importing libraries for data reading & manipulation.

In [27]:
import pandas as pd
import numpy as np
import random
import math
import sys

Reading input filename and attribute splitting criteria from command line argument.

In [28]:
try:
    file =  str(sys.argv[1])
    method = str(sys.argv[2])
    
except:
    print ("\nPlease enter the correct command line arguments:")
    print ("./decision_tree.py <Train_file name> <method(Gini/Igain/both)> ")
    sys.exit()
file = 'data_banknote_authentication.csv'
method = 'both'

Setting default depth to 10 and extracting the name of label column.

In [29]:
data = pd.read_csv(file)
depth = 10
label = data.columns[-1]

Code to randomly split the training data into train & test data.

In [30]:
def cross_validation(train_size,data):
    train_data = data.sample(frac=train_size)
    test_data = data.drop(train_data.index)
    return train_data.reset_index(drop=True),test_data.reset_index(drop=True)

train_data,test_data = cross_validation(0.7,data)
classes = data[label].unique()


Function code to split the data based on particular column and column value

In [31]:
def split_data(column,value,data):
    left_child = data[data[column]<=value]
    right_child = data[data[column]>value]
    return left_child,right_child

Function code to compute the best splitting value for a column using Gini

In [33]:
def best_split_value(column,data):
    current_tree = data
    values = current_tree[column]
    values = sorted(values.unique())
    check_values = []
    gini_arr = []
    for i in range(len(values)):
        if i+1<len(values):
            mean = values[i]+values[i+1] 
            check_values.append(mean/2)
        if len(values) == 1:
            check_values.append(values[0])
    for every_value in check_values:
        left_child,right_child = split_data(column,every_value,train_data)
        gini_index = calc_gini(classes,left_child,right_child,data,label)
        gini_arr.append((gini_index,every_value))
    best_gini = (min(gini_arr))
    return best_gini


Function code to find the majority class label for records in leaf nodes.

In [35]:
def leaf_nodes(current_node):
    count_groupby_class = (current_node.groupby(label).size())
    return count_groupby_class.idxmax(),count_groupby_class.max()


Function code to classify the test data using the decision tree constructed.

In [36]:
def classify(root,testline):
    if testline[root['best_attr']]<= root['best_attr_val']:
        if isinstance(root['left_tree'],dict):
            return classify(root['left_tree'],testline)
        else:
            return root['left_tree']
    else:
        if isinstance(root['right_tree'],dict):
            return classify(root['right_tree'],testline)
        else:
            return root['right_tree']


Function code to calculate the gini value for any particular split.

In [32]:
def calc_gini(classes,left_node,right_node,current_node,label):
    
    total_nodes = len(current_node)
    no_left_nodes = len(left_node)
    no_right_nodes = len(right_node)
    
    left_gini =0.0
    right_gini = 0.0
    
    total_prob = 0.0
    
    for each_class in classes:
        if no_left_nodes==0:
            continue
        prob = len(left_node[(left_node[label]==each_class)])/float(no_left_nodes)
        total_prob += prob**2
    left_gini = 1-total_prob
    
    total_prob = 0.0
    for each_class in classes:
        if no_right_nodes==0:
            continue
        prob = len(right_node[(right_node[label]==each_class)])/float(no_right_nodes)
        total_prob += prob**2
    right_gini = 1-total_prob
    
    weighted_gini = ((float(no_left_nodes)/total_nodes)*float(left_gini)) + ((float(no_right_nodes)/total_nodes)*float(right_gini))
    return round(weighted_gini,6)



Function code to calculate the entropy for any particular split.

In [37]:
def calc_entropy(left_nodes,right_nodes,current_node,label):
    N = len(current_node)
    n1 = len(left_nodes)
    n2 = len(right_nodes)
    
    if n1!=0:
        entropy1 = 0.0
        for each_class in classes:
            n1_class = len(left_nodes[left_nodes[label] == each_class])
            #print(n1_class)
            if(n1_class) == 0:
                value = 0.0
                continue
            value = -float(n1_class)/n1 * math.log(float(n1_class)/n1,2.0)
            #print(value)
            entropy1+=value
    
    entropy2 = 0.0
    if n2!=0:
        for each_class in classes:
            n2_class = len(right_nodes[right_nodes[label] == each_class])
            if(n2_class) == 0:
                value = 0.0
                continue
            value = -float(n2_class)/n2 * math.log(float(n2_class)/n2,2.0)
            entropy2+=value
    entropy = float(n1)/N*(entropy1) + float(n2)/N*(entropy2)
    return entropy


Function code to compute the best splitting value for a column using information gain

In [38]:
def calc_infogain(curr_data,column,parent_entropy):
    current_tree = curr_data
    values = current_tree[column]
    values = sorted(values.unique())
    check_values = []
    infogain = []
    for i in range(len(values)):
        if i+1<len(values):
            mean = values[i]+values[i+1] 
            check_values.append(mean/2)
        if len(values) == 1:
            check_values.append(values[0])
            
    for every_value in check_values:
        left_child,right_child = split_data(column,every_value,train_data)
        entropy = calc_entropy(left_child,right_child,data,label)
        infogain.append(((parent_entropy-entropy),every_value,entropy))
        
    best_infogain = max(infogain)
    return best_infogain

Function code to find the best attribute for split using gini.

In [34]:
def best_attribute(data,columns):

    node ={}
    node['best_attr'] = ""
    node['best_attr_val'] = 999
    node['groups'] = None
    gini = 999
    if len(columns)==0:
        return 
    for each_column in columns:
        current_gini = best_split_value(each_column,data)
        if current_gini[0]<=gini:
            gini = current_gini[0]
            best_attr = each_column
            best_attr_val = current_gini[1]    
    children = split_data(column=best_attr,value=best_attr_val,data = data)
    node['best_attr'] = best_attr
    node['best_attr_val'] = best_attr_val
    node['children'] = children   
    return (node)


Function code to find the best attribute for split using information gain.

In [39]:
def best_attribute_entropy(columns,data,parent_infogain):
 
    node ={}
    node['best_attr'] = ""
    node['best_attr_val'] = 99999
    node['children'] = None
    infogain = -999999
    if len(columns)==0:
        return 
    for each_column in columns:
        current_igain = calc_infogain(data,each_column,parent_infogain)
        if current_igain[0]>infogain:
            infogain = current_igain[0]
            best_attr = each_column
            best_attr_val = current_igain[1]
            best_entropy = current_igain[2]
    children = split_data(column=best_attr,value=best_attr_val,data = data)
    node['best_attr'] = best_attr
    node['best_attr_val'] = best_attr_val
    node['children'] = children   
    return (node,best_entropy)


Function code to expand the tree recursively untill the stopping criteria isn't satisfied using gini.

In [40]:
def expand_tree(node,max_depth,current_depth,columns):

    left_tree, right_tree = node['children']
    best_attr = node['best_attr']
    del(node['children'])
    frames = [left_tree,right_tree]
    result = pd.concat(frames)
    if(result[label].nunique())==1:
        node['left_tree'] = node['right_tree'] = list(result[label].unique())[0]
        return
    
    if left_tree.empty or right_tree.empty:     
        node['left_tree'] = node['right_tree'] = leaf_nodes(result)[0]
        return 

    if len(columns) ==1 or current_depth>=max_depth:
        node['left_tree'] = leaf_nodes(left_tree)[0]
        node['right_tree'] = leaf_nodes(right_tree)[0]
        return
    
    columns = columns.drop(best_attr)
    node['left_tree'] = best_attribute(left_tree,columns) 
    expand_tree(node['left_tree'],max_depth,current_depth+1,columns)

    node['right_tree'] = best_attribute(right_tree,columns)
    expand_tree(node['right_tree'],max_depth,current_depth+1,columns)


Function code to expand the tree recursively untill the stopping criteria isn't satisfied using gini.

In [41]:
def expand_tree_igain(node,max_depth,current_depth,columns,parent_igain):

    left_tree, right_tree = node['children']
    best_attr = node['best_attr']
    del(node['children'])
    frames = [left_tree,right_tree]
    result = pd.concat(frames)
    if(result[label].nunique())==1:
        node['left_tree'] = node['right_tree'] = list(result[label].unique())[0]
        return
    
    if left_tree.empty or right_tree.empty:     
        node['left_tree'] = node['right_tree'] = leaf_nodes(result)[0]
        return 

    if len(columns) ==1 or current_depth>=max_depth:
        node['left_tree'] = leaf_nodes(left_tree)[0]
        node['right_tree'] = leaf_nodes(right_tree)[0]
        return
    
    columns = columns.drop(best_attr)
    node['left_tree'],child_igain = best_attribute_entropy(columns,left_tree,parent_igain) 
    expand_tree_igain(node['left_tree'],max_depth,current_depth+1,columns,child_igain)

    node['right_tree'],child_igain = best_attribute_entropy(columns,right_tree,parent_igain)
    expand_tree_igain(node['right_tree'],max_depth,current_depth+1,columns,child_igain)



Code to build decision tree depending on the selected splitting criteria.

In [42]:

if method!='gini' and method!='igain' and method!='both':
    print ("Please enter the correct method(gini/igain/both)")
    sys.exit()

if method=='gini' or method=='both':
    print ("Training decision tree using gini.")
    root1 = best_attribute(train_data,train_data.columns.drop(label))
    expand_tree(root1,depth,1,train_data.columns.drop(label))

if method == 'igain' or method == 'both':
    print ("Training decision tree using igain.")
    counts = train_data[label].value_counts()
    val = 0.0
    root_igain = 0.0
    for each in counts:
        root_igain += -each/len(train_data) * math.log(float(each)/len(train_data),2.0)
    root2,child_gain = best_attribute_entropy(train_data.columns.drop(label),train_data,root_igain)
    expand_tree_igain(root2,depth,1,train_data.columns.drop(label),child_gain)


Training decision tree using gini.
Training decision tree using igain.


In [43]:
gini_count = 0.0
igain_count = 0.0
print ("Classification in progress.")
for i in range(0,len(train_data)):
    truth = train_data.iloc[i][label]
    if method == 'gini' or method == 'both':
        prediction1 = classify(root1,train_data.iloc[i])
        if prediction1 == truth :
            gini_count +=1.0
    if method == 'igain' or method == 'both':
        prediction2 = classify(root2,train_data.iloc[i])
        if prediction2 == truth :
            igain_count +=1.0

if method == 'gini':
    print ("Accuracy using", method,":",round((gini_count/len(train_data)*100),4))
elif method == 'igain':
    print ("Accuracy using", method,":",round((igain_count/len(train_data)*100),4))
elif method == 'both':
    print ("Accuracy using", method,":")
    print ("gini:",round((gini_count/len(train_data)*100),4))
    print ("igain:",round((igain_count/len(train_data)*100),4))

Classification in progress.
Accuracy using both :
gini: 89.8958
igain: 89.375
